In [1]:

import keras
import numpy as np
import os
import cv2

import csv
import pandas as pd
import pathlib
import fnmatch



import ssl

### To avoid certificat error (source: https://stackoverflow.com/questions/27835619/urllib-and-ssl-certificate-verify-failed-error)
import requests
requests.packages.urllib3.disable_warnings()

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context


from keras.applications import inception_resnet_v2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt



from keras.preprocessing import image
from keras.applications import vgg16
import numpy as np



Using TensorFlow backend.


In [2]:

!mkdir -p googledrive
from google.colab import drive
drive.mount('/content/googledrive')
 




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/googledrive


In [0]:
default_path = '/content/googledrive/My Drive/FlickrCNN'
os.chdir(default_path)
photo_path = default_path + '/Photos_338_retraining'



In [0]:
from keras.applications import inception_resnet_v2, inception_v3


from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import Adam

from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
img_width, img_height = 331, 331
train_data_dir = "Photos_338_retraining/train"
validation_data_dir = "Photos_338_retraining/validation"
nb_train_samples = 210
nb_validation_samples = 99

batch_size = 32 # proportional to the training sample size..
epochs = 50

num_classes = 4


In [7]:
##### build our classifier model based on pre-trained InceptionResNetV2:

# Load the base pre-trained model

# do not include the top fully-connected layer
# 1. we don't include the top (fully connected) layers of InceptionResNetV2



 
# Load the base pre-trained model
# do not include the top fully-connected layer
model = inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))

# New dataset is small and similar to original dataset:
# There is a problem of over-fitting, if we try to train the entire network. Since the data is similar to the original data, we expect higher-level features in the ConvNet to be relevant to this dataset as well. Hence, the best idea might be to train a linear classifier on the CNN codes.
# So lets freeze all the layers and train only the classifier
 
# Freeze the layers which you don't want to train. Here I am freezing the all layers.
# i.e. freeze all InceptionV3 layers
for layer in model.layers[:]:
    layer.trainable = False


87916544/87910968 [==============================] - 2s 0us/step


In [0]:

# Adding custom Layer
x = model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)
 
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have n classes
predictions = Dense(num_classes, activation='softmax')(x)
 
 
# creating the final model
# this is the model we will train
model_final = Model(inputs = model.input, outputs = predictions)



# creating the final model
# this is the model we will train
model_final = Model(inputs = model.input, outputs = predictions)


#Now we will be training only the classifiers (FC layers)

# compile the model (should be done *after* setting layers to non-trainable)
# Compile the final model using an Adam optimizer, with a low learning rate (since we are 'fine-tuning')
model_final.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
# model_final.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model_final.compile(loss='sparse_categorical_crossentropy',
#               optimizer=Adam(lr=0.0001),
#               metrics=['acc'])

# print(model_final.summary())



In [9]:


# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = "categorical")
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical")




Found 210 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


In [10]:
# show class indices
print('****************')
for cls, idx in train_generator.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')



****************
Class #0 = backpacking
Class #1 = hiking
Class #2 = hotsprings
Class #3 = noactivity
****************


In [0]:
# Save the model according to the conditions
checkpoint = ModelCheckpoint("InceptionV3_retrain.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')







In [0]:


# Re-train the model
history = model_final.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples,
    callbacks = [checkpoint, early])

# at this point, the top layers are well trained.





Epoch 1/50
  2/210 [..............................] - ETA: 1:33:08 - loss: 1.6881 - acc: 0.1250

In [0]:
# at this point, the top layers are well trained.
# Let's save the model
model_final.save('InceptionV3_retrain_instagram_final.h5')

In [0]:
 

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [0]:
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
  

In [0]:

errors = np.where(np.not_equal(predicted_classes, ground_truth[0]))
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 

In [0]:

# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_data_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()